# NLP (Natural Language Processing) with Python

### Imports

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

## Data

In [ ]:
messages = [line.rstrip() for line in open('/home/jovyan/datafabric/tutorial/spam_utf8.csv')] #please update the path.
print(len(messages))

In [ ]:
for message_no, message in enumerate(messages[:3]):
    print(message_no, message)
    print('\n')

In [ ]:
messages = pd.read_csv('/home/jovyan/datafabric/tutorial/spam_utf8.csv', sep=',', #please update the path.
                           names=["label", "message", "v3", "v4", "v5"])
messages.head()

## Exploratory Data Analysis

In [ ]:
messages.describe()

In [ ]:
messages.groupby('label').describe()

In [ ]:
messages['length'] = messages['message'].apply(len)
messages.head()

In [ ]:
messages

### Data Visualization

In [ ]:
messages['length'].plot(bins=50, kind='hist');

In [ ]:
messages.length.describe()

In [ ]:
messages.hist(column='length', by='label', bins=50,figsize=(12,4));

## Text Pre-processing

In [ ]:
mess = 'Sample message! Notice: it has punctuation.'

# Check characters to see if they are in punctuation
nopunc = [char for char in mess if char not in string.punctuation]

# Join the characters again to form the string.
nopunc = ''.join(nopunc)

In [ ]:
stopwords.words('english')[0:10] # Show some stop words

In [ ]:
nopunc.split()

In [ ]:
# Now just remove any stopwords
clean_mess = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
clean_mess

In [ ]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages.head()

In [ ]:
# Check to make sure its working
messages['message'].head(5).apply(text_process)

In [ ]:
# Show original dataframe
messages.head()

## Vectorization

In [ ]:
# Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

In [ ]:
message4 = messages['message'][3]
print(message4)

In [ ]:
bow4 = bow_transformer.transform([message4])
print(bow4)
print(bow4.shape)

In [ ]:
print(bow_transformer.get_feature_names_out()[4073])
print(bow_transformer.get_feature_names_out()[9570])

In [ ]:
messages_bow = bow_transformer.transform(messages['message'])

In [ ]:
print('Shape of Sparse Matrix: ', messages_bow.shape)
print('Amount of Non-Zero occurences: ', messages_bow.nnz)

In [ ]:
sparsity = (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

In [ ]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

In [ ]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['u']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['university']])

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

## Training a model

In [ ]:
spam_detect_model = MultinomialNB().fit(messages_tfidf, messages['label'])

In [ ]:
print('predicted:', spam_detect_model.predict(tfidf4)[0])
print('expected:', messages.label[3])

## Model Evaluation

In [ ]:
all_predictions = spam_detect_model.predict(messages_tfidf)
print(all_predictions)

In [ ]:
print (classification_report(messages['label'], all_predictions))

## Train Test Split

In [ ]:
msg_train, msg_test, label_train, label_test = \
train_test_split(messages['message'], messages['label'], test_size=0.2)

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

## Creating a Data Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
pipeline.fit(msg_train,label_train)

In [ ]:
predictions = pipeline.predict(msg_test)

In [ ]:
print(classification_report(predictions,label_test))